In [ ]:
import requests
from slacker import Slacker
from config import slack_cnf
import random
import requests
import json


# Sending message to Slack
# Regarding the library: Slacker: https://github.com/os/slacker/blob/master/slacker/__init__.py
# Regarding incoming-webhooks: https://api.slack.com/incoming-webhooks


def send_post(message, channel='___gen___'):
    post_url = 'https://slack.com/api/chat.postMessage'
    headers = {"Authorization": "Bearer " + slack_cnf.access_token}
    payload = {
        'channel': channel,
        "text": message,
        'as_user': False
    }
    resp = requests.post(post_url, headers=headers, data=payload)
    print(resp.json())
    return resp


def send_post_with_thread(thread_text, message_list, channel=None, attachments=None):
    post_url = 'https://slack.com/api/chat.postMessage'
    headers = {"Authorization": "Bearer " + slack_cnf.access_token}
    payload = {
        'channel': channel,
        "text": thread_text
    }
    resp = requests.post(post_url, headers=headers, data=payload)
    print(resp.json())
    thread_ts = resp.json()['ts']
    for message in message_list:
        payload = {
            'channel': channel,
            "text": message,
            'thread_ts': thread_ts
        }
        resp = requests.post(post_url, headers=headers, data=payload)

    return resp


def notify(text, channel=None, attachments=None, slack_webhook_url=None):
    """
    Text Cheatsheet:
    Mention: <@> -> ユーザーIDはポイント
    Emoji: :smile:
    :param text:
    :param channel:
    :param attachments:
    :param slack_webhook_url:
    :return:
    """
    if slack_webhook_url is None:
        slack_webhook_url = slack_cnf.slack_webhook_url[channel]
    if channel is None:
        channel = slack_cnf.default_channel

    slack = Slacker('', incoming_webhook_url=slack_webhook_url)
    response = slack.incomingwebhook.post({
        'channel': channel,
        'text': text,
        'link_names': '1',
        'attachments': attachments
    })



def notify_with_thread(thread_text, message_list, channel=None, attachments=None, slack_webhook_url=None):
    """
    Post response:
    https://api.slack.com/methods/chat.postMessage
    Thread:
    :param thread_text:
    :param message_list:
    :param channel:
    :param attachments:
    :param slack_webhook_url:
    :return:
    """
    if slack_webhook_url is None:
        slack_webhook_url = slack_cnf.slack_webhook_url[channel]
    if channel is None:
        channel = slack_cnf.default_channel
    print(slack_webhook_url)
    slack = Slacker('', incoming_webhook_url=slack_webhook_url)
    thread_response = slack.incomingwebhook.post({
        'channel': channel,
        'text': thread_text,
        'link_names': '1',
        'attachments': attachments
    })
    print(thread_response.__repr__)
    print(thread_response.raw.__dict__)
    thread_json = thread_response.json()

    parent_ts = thread_json["ts"]
    for message_text in message_list:
        thread_response = slack.incomingwebhook.post({
            'channel': channel,
            'thread_ts': parent_ts,
            'text': message_text,
            'link_names': '1',
            'attachments': attachments
        })
        json = thread_response.json()

    return thread_json